In [1]:
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt
import sys, os
sys.path.append('../..')
from lcs.plotting_utils import *
from lcs import TEXTWIDTH, PAGEHEIGHT
import numpy as np
from lcs_mnist.mnist_lcs_src import indicate_contexts
from lcs_mnist.mnist_lcs_plotting_utils import read_in_data, indicate_contexts, plot_wcst, plot_specificity
from lcs_mnist import mnist_learning_clean
from lcs.configs import *
from lcs.joint_learning import *
import pickle

In [ ]:
# TODO: MAKE THIS EASY TO RUN FROM THIS NOTEBOOK

In [3]:
data_timestamp = 20240513175956 #, 20240802131757

In [4]:
cfg = Config(**{
            'input_size': 64,
            'output_size': 10,
            'num_seeds': 10,
            'num_contexts': 2,
            'num_paths': 2,
            'batch_size': 100,
            'W_lr': 0.001,
            'c_lr': 2,
            'initialization_scale': 0.01,
            'num_blocks': 10,
            'block_duration': 100,
            'regularization_strength': 0.1,
            'num_layers': 1,
            'hidden_size': 32,
            'context_model': True,
            'name': 'mnist_cfg',
            'regularization_type': ['gating_manifold_L1', 'nonnegative'],
            'shared_teachers': False,
            'c_gt_curriculum': 'A_B__',
            'dt': 0.1,
            'T_tape': -1,
            'dataset_name': 'mnist',
            'data_appendix': '_CNN_bottleneck10',
            'permutation1': None,
            'permutation2': None,
            'data_folder': os.path.join('data', 'mnist'),
            'results_folder': os.path.join('results', 'mnist', "%s_%s"%(str(None), str(None)))
        })

[WARNING] [2025-02-19 20:21:34,026] in lcs.configs: Using a dt > 1e-3 introduces finite-size effects!


In [5]:
mnist_learning_clean.run_cfg(cfg, data_timestamp)

initializing...


/Users/alex/anaconda3/envs/NTA/lib/python3.13/site-packages/jax/_src/numpy/lax_numpy.py:6944: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  start = asarray(start, dtype=computation_dtype)
/Users/alex/anaconda3/envs/NTA/lib/python3.13/site-packages/jax/_src/numpy/lax_numpy.py:6945: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  stop = asarray(stop, dtype=computation_dtype)
/Users/alex/anaconda3/envs/NTA/lib/python3.13/site-packages/jax/_src/numpy/lax_numpy.py:6954: UserWarning:

FileNotFoundError: [Errno 2] No such file or directory: 'data/mnist/20240513175956_ys_train_CNN_bottleneck10.npy'

In [ ]:
forget_cfg = Config(**{
            'input_size': 64,
            'output_size': 10,
            'num_seeds': 10,
            'num_contexts': 2,
            'num_paths': 2,
            'batch_size': 100,
            'W_lr': 0.001,
            'c_lr': 0.001,
            'initialization_scale': 0.01,
            'num_blocks': 10,
            'block_duration': 100,
            'regularization_strength': np.finfo(float).eps,
            'num_layers': 1,
            'hidden_size': 32,
            'context_model': True,
            'name': 'mnist_cfg',
            'regularization_type': ['gating_manifold_L1', 'nonnegative'],
            'shared_teachers': False,
            'c_gt_curriculum': 'A_B__',
            'dt': 0.1,
            'T_tape': -1,
            'dataset_name': 'mnist',
            'data_appendix': '_CNN_bottleneck10',
            'permutation1': None,
            'permutation2': None,
            'data_folder': os.path.join('data', 'mnist'),
            'results_folder': os.path.join('results', 'mnist', "%s_%s"%(str(None), str(None)))
        })

In [3]:
init_plot_settings()
s = 2
fig = plt.figure(figsize=(s*TEXTWIDTH, s*PAGEHEIGHT*0.125), layout='constrained')

mosaic = """
ab
"""

axd = fig.subplot_mosaic(mosaic)

ax_acc = axd['a']
ax_c = axd['b']

In [4]:
run_timestamps = ['20241017133547', '20241017125926'] ## L2 Regularization
dataset_name = 'mnist'

### NEW fMNIST RUNS - WARM COLD, truly shared structure
#run_timestamps = ['20241030235955', '20241031000021']
#dataset_name = 'fashion_mnist'

labels = ['flexible' , 'forgetful']

#permutation_names = "None_warm_cool"
permutation_names = "None_upper_lower"
#permutation_names = None

results_folder = os.path.join('..', 'results', dataset_name)

if permutation_names is not None:
    results_folder = os.path.join(results_folder, permutation_names)

os.makedirs(results_folder, exist_ok=True)

ts, losses, cs, norm_W1s, accs, contexts, cfgs = [], [], [], [], [], [], []

for run_timestamp, label in zip(run_timestamps, labels):

    t, loss, c, norm_W1, acc, context, cfg = read_in_data(run_timestamp, results_folder = results_folder, label=None)
    ts.append(t)
    losses.append(loss)
    cs.append(c)
    norm_W1s.append(norm_W1)
    accs.append(acc)
    contexts.append(context)
    cfgs.append(cfg)

model_colors = ['black', 'grey']

t = ts[0]
loss = losses[0]
c = cs[0]
norm_W1 = norm_W1[0]
acc = accs[0]
context = contexts[0]
cfg = cfgs[0]

accs = np.array(accs)
ts = np.array(ts)
c = np.array(c)

FileNotFoundError: [Errno 2] No such file or directory: '../results/mnist/None_upper_lower/20241017133547_run_info.pkl'

In [ ]:
ax = ax_acc

for i in range(len(ts)):
    means = np.mean(accs[i], axis=0)
    stderrs = np.std(accs[i], axis=0) / np.sqrt(len(accs[i]))

    #fill_between(ax, ts[i], accs[i], label=labels[i], c=model_colors[i]) 
    ax.plot(ts[i, 0], means, label=labels[i], c=model_colors[i])
    ax.fill_between(ts[i, 0], means - stderrs, means + stderrs, alpha=0.3, color=model_colors[i])

ax.dataLim.y0 = 0
ax.dataLim.y1 = 1

ax.set_ylabel('accuracy')
ax.set_xlabel('time $t$')
ax.autoscale_view()
ax.set_xlim((min(ts[0,0]), max(ts[0,0])))

ax.legend()

indicate_contexts(ax, t, cfg)

In [ ]:
ax = ax_c

sorted_gates = []

for i in range(len(c)):
    current_gates = c[i]
    
    ## sort based on final 100 timesteps
    sort_idx = np.argsort(np.mean(current_gates[-100:], axis=0))
    sorted_gates.append(current_gates[:, sort_idx])
    
means = np.mean(sorted_gates, axis=0)
stderrs = np.std(sorted_gates, axis=0) / np.sqrt(len(current_gates))

for p in range(cfg.num_paths):
    ax.plot(t[0], means[..., p], label=f'c{p+1}', c=f'C{p}')
    ax.fill_between(t[0], means[..., p] - stderrs[..., p], means[..., p] + stderrs[..., p], alpha=0.3, color=f'C{p}')

c_min = np.min(c)
c_max = np.max(c)
ax.dataLim.y0 = c_min - limpad
ax.dataLim.y1 = c_max + limpad
ax.set_yticks([0, .5, 1, 1.5, 2, 2.5])
ax.autoscale_view()
ax.set_xlim((min(ts[0,0]), max(ts[0,0])))

#ax.legend()

ax.set_xlabel(r'time $t$')
ax.set_ylabel(r'gates $c^p$')

indicate_contexts(ax, t, cfg)

fig